In [ ]:
from tqdm import tqdm
from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import configure
import gymnasium as gym
import numpy as np
import random
import csv

In [ ]:
class ProgressBarCallbackWithFileLogging(BaseCallback):
    def __init__(self, total_timesteps, log_file='training_log.csv', reward_log_file='episode_rewards_log.csv', verbose=0):
        super(ProgressBarCallbackWithFileLogging, self).__init__(verbose)
        self.total_timesteps = total_timesteps
        self.pbar = None
        self.episode_rewards = []  # Suivi des récompenses par épisode
        self.max_reward = -np.inf  # Récompense maximale
        self.current_episode_reward = 0  # Récompense cumulée pour l'épisode en cours
        self.episode_count = 0  # Compteur d'épisodes
        self.log_file = log_file   # Nom du fichier de log des statistiques globales
        self.reward_log_file = reward_log_file  # Nom du fichier de log des récompenses par épisode

        # Initialiser le fichier CSV pour les stats générales
        with open(self.log_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Timestep', 'Mean Reward (1000 steps)', 'Max Reward', 'Actor Loss', 'Critic Loss'])

        # Initialiser le fichier CSV pour les récompenses par épisode
        with open(self.reward_log_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Episode', 'Reward'])

    def _on_training_start(self):
        """Initialisation de la barre de progression au début de l'entraînement."""
        print("Entraînement commencé...")
        self.pbar = tqdm(total=self.total_timesteps)

    def _on_step(self):
        """Mise à jour de la barre de progression et écriture des infos dans les fichiers."""
        # Mise à jour de la barre de progression
        self.pbar.update(1)

        # Ajouter la récompense courante à l'épisode en cours
        current_reward = self.locals['rewards'][0]
        self.current_episode_reward += current_reward

        # Mettre à jour la récompense maximale si nécessaire
        if current_reward > self.max_reward:
            self.max_reward = current_reward

        # Si l'épisode se termine, enregistrer la récompense de l'épisode
        if self.locals['dones'][0]:
            self.episode_count += 1

            # Enregistrer la récompense cumulée de l'épisode dans le fichier CSV
            with open(self.reward_log_file, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([self.episode_count, self.current_episode_reward])

            # Réinitialiser la récompense pour le prochain épisode
            self.current_episode_reward = 0

        # Enregistrer des informations toutes les 1000 étapes
        if self.num_timesteps % 1000 == 0:
            mean_reward = np.mean(self.episode_rewards[-1000:]) if len(self.episode_rewards) >= 1000 else np.mean(self.episode_rewards)
            actor_loss = self.locals.get('actor_loss', None)
            critic_loss = self.locals.get('critic_loss', None)

            # Écriture des statistiques globales dans le fichier CSV
            with open(self.log_file, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([self.num_timesteps, mean_reward, self.max_reward, actor_loss, critic_loss])

        return True

    def _on_training_end(self):
        """Fermer la barre de progression et terminer le logging."""
        self.pbar.close()
        print(f"Entraînement terminé. Les informations sont enregistrées dans {self.log_file}")
        print(f"Récompense maximale atteinte: {self.max_reward:.2f}")

In [ ]:
# Paramètres communs
n_actions = 2
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

# Paramètres spécifiques TD3
learning_rate = 1e-3
batch_size = 64
buffer_size = int(2e5)
tau = 0.05
gamma = 0.98
learning_starts = 10000
n_steps = 100
total_timesteps = n_steps * 1_000  # Réduire à 100k pour tester rapidement
num_episodes_eval = 10
policy_kwargs = dict(
    net_arch=dict(pi=[400, 300], qf=[400, 300])  # Architecture des réseaux d'acteur et de critique
)

In [ ]:
# Initialiser les résultats
seeds = [random.randint(0, 10000) for _ in range(10)]  # Générer 5 seeds aléatoires
rewards_per_seed = {}  # Dictionnaire pour stocker les récompenses par seed

# Configurer TensorBoard pour logger l'entraînement
new_logger = configure("./logs/", ["tensorboard"])

In [ ]:
# Boucle sur chaque seed
for seed in seeds:
    print(f"Training with seed {seed}")
    
    # Créer l'environnement avec la seed
    env = gym.make('LunarLanderContinuous-v2')
    env.reset(seed=seed)

    # Initialiser le modèle TD3 avec des hyperparamètres personnalisés
    model = TD3(
        "MlpPolicy",
        env,
        action_noise=action_noise,
        learning_rate=learning_rate,
        batch_size=batch_size,
        buffer_size=buffer_size,
        tau=tau,
        gamma=gamma,
        train_freq=n_steps,
        gradient_steps=-1,
        learning_starts=learning_starts,
        policy_kwargs=policy_kwargs,
        verbose=1,
        seed=seed,
        device="cuda"  # Utiliser le GPU si disponible
    )

    # Assigner le nouveau logger au modèle
    model.set_logger(new_logger)

    # Créer et ajouter le callback de progression
    progress_callback = ProgressBarCallbackWithFileLogging(total_timesteps, log_file=f"training_log_seed_{seed}.csv", reward_log_file=f"episode_rewards_log_seed_{seed}.csv")

    # Entraîner le modèle
    model.learn(total_timesteps=total_timesteps, callback=progress_callback)

    # Sauvegarder le modèle après l'entraînement pour chaque seed (optionnel)
    model.save(f"td3_lunar_lander_seed_{seed}")

    # Évaluation du modèle après l'entraînement
    total_rewards = []
    for episode in range(num_episodes_eval):
        obs, info = env.reset()
        done = False
        episode_reward = 0
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, info = env.step(action)
            episode_reward += reward
            done = terminated or truncated
        total_rewards.append(episode_reward)

    # Calculer et stocker la moyenne des récompenses pour cette seed
    mean_reward = np.mean(total_rewards)
    rewards_per_seed[seed] = mean_reward
    print(f"Mean reward for seed {seed}: {mean_reward}")

In [ ]:
# Afficher les récompenses moyennes pour chaque seed
print("\nRewards per seed:")
for seed, reward in rewards_per_seed.items():
    print(f"Seed {seed}: Mean Reward = {reward}")

# Calculer la récompense moyenne sur les 10 seeds
overall_mean_reward = np.mean(list(rewards_per_seed.values()))
print(f"\nOverall mean reward across all seeds: {overall_mean_reward}")